# Playing around with the Spotify API

In [32]:
import pandas as pd
import numpy as np
import os
import sys
import spotipy
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

from scipy import spatial

In [18]:

os.environ['SPOTIPY_CLIENT_ID'] = "cba9273cc2784d68b10cab9a7f7a25e6" 
os.environ['SPOTIPY_CLIENT_SECRET'] = "5df1e92959334eaca52ccdcb279f31c6" 
os.environ['SPOTIPY_REDIRECT_URI'] = "http://localhost:8888/callback" 

#export SPOTIPY_CLIENT_ID='cba9273cc2784d68b10cab9a7f7a25e6'
#!export SPOTIPY_CLIENT_SECRE='5df1e92959334eaca52ccdcb279f31c6'T
#!export SPOTIPY_REDIRECT_URI='http://localhost:8888/callback'

In [20]:
'''
define the API connection - usually log in from here as well...
'''
token = util.prompt_for_user_token('cba9273cc2784d68b10cab9a7f7a25e6', '')

client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


#spotify=spotipy.Spotify()




            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=cba9273cc2784d68b10cab9a7f7a25e6&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback in your browser


Enter the URL you were redirected to: http://localhost:8888/callback?code=AQD2v-vlTAkWD7oMfT6HjR9mBEwAOURhiwG2IogHtRzjW9iP2ONDPEtaV3Zw3v3mLnEAwScRzAaWrkBD9WQOHvxyUzAl438Ug4BzUY6NsaVR5El6U774zZAJZu6i-OYJ_QMYm31Xxhy-1wH0m3URmDO4aFZ0ElDroIKoo8Xpg5bZ4BNYpUzkdd06e5Vf6T5_vJccMw




In [49]:
'''example with finding a users albums... '''
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP' #try and make a function
#that returns this 'uri' given artist name.

results = sp.artist_top_tracks(birdy_uri, country = 'SE')
#albums = results['items']
print(results)

#while results['next']:
#    results = spotify.next(results)
#    albums.extend(results['items'])

#for album in albums:
#    print(album['name'])

{'tracks': [{'artists': [{'type': 'artist', 'id': '2WX2uTcsvV5OnS0inACecP', 'name': 'Birdy', 'external_urls': {'spotify': 'https://open.spotify.com/artist/2WX2uTcsvV5OnS0inACecP'}, 'href': 'https://api.spotify.com/v1/artists/2WX2uTcsvV5OnS0inACecP', 'uri': 'spotify:artist:2WX2uTcsvV5OnS0inACecP'}], 'popularity': 78, 'track_number': 2, 'name': 'Skinny Love', 'id': '4RL77hMWUq35NYnPLXBpih', 'href': 'https://api.spotify.com/v1/tracks/4RL77hMWUq35NYnPLXBpih', 'uri': 'spotify:track:4RL77hMWUq35NYnPLXBpih', 'type': 'track', 'album': {'artists': [{'type': 'artist', 'id': '2WX2uTcsvV5OnS0inACecP', 'name': 'Birdy', 'external_urls': {'spotify': 'https://open.spotify.com/artist/2WX2uTcsvV5OnS0inACecP'}, 'href': 'https://api.spotify.com/v1/artists/2WX2uTcsvV5OnS0inACecP', 'uri': 'spotify:artist:2WX2uTcsvV5OnS0inACecP'}], 'id': '1WGjSVIw0TVfbp5KrOFiP0', 'album_type': 'album', 'type': 'album', 'available_markets': ['AR', 'AT', 'BE', 'BG', 'BO', 'BR', 'CH', 'CL', 'CO', 'CR', 'CZ', 'DE', 'DK', 'DO', '

In [24]:
''' 
luke i changed ur function down below the index 1 was wrong so i put it as  0 and it giving
way better results, chur...
'''
def get_uri(artist):
    results = sp.search(q = "artist:" + artist, type = "artist")
    artist_id = results['artists']['items'][0]['uri'] # changed from '1' to '0' ??
   
    return(artist_id)

get_uri("Beyonce")
#A function that returns artist 'uri' as a string. 
#By most matching (first result)

'spotify:artist:6vWDO969PvNqNYHIOW5v0m'

In [54]:
sp.categories()

{'categories': {'href': 'https://api.spotify.com/v1/browse/categories?offset=0&limit=20',
  'items': [{'href': 'https://api.spotify.com/v1/browse/categories/toplists',
    'icons': [{'height': 275,
      'url': 'https://t.scdn.co/media/derived/toplists_11160599e6a04ac5d6f2757f5511778f_0_0_275_275.jpg',
      'width': 275}],
    'id': 'toplists',
    'name': 'Top Lists'},
   {'href': 'https://api.spotify.com/v1/browse/categories/mood',
    'icons': [{'height': 274,
      'url': 'https://t.scdn.co/media/original/mood-274x274_976986a31ac8c49794cbdc7246fd5ad7_274x274.jpg',
      'width': 274}],
    'id': 'mood',
    'name': 'Mood'},
   {'href': 'https://api.spotify.com/v1/browse/categories/party',
    'icons': [{'height': 274,
      'url': 'https://t.scdn.co/media/links/partyicon_274x274.jpg',
      'width': 274}],
    'id': 'party',
    'name': 'Party'},
   {'href': 'https://api.spotify.com/v1/browse/categories/pop',
    'icons': [{'height': 274,
      'url': 'https://t.scdn.co/media/deri

In [51]:
''' lets try get related song '''

artst = 'Young Thug'

# get audio analysis of SP track skinny love...
related = sp.audio_features('spotify:track:4RL77hMWUq35NYnPLXBpih')

print(related)
#for artist in related['artists']:
    #print('  ', artist['name'])
    #print('  ', artist['popularity'])
    #print('  ', artist['followers']['total'])



[{'track_href': 'https://api.spotify.com/v1/tracks/4RL77hMWUq35NYnPLXBpih', 'tempo': 166.467, 'speechiness': 0.051, 'type': 'audio_features', 'mode': 1, 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4RL77hMWUq35NYnPLXBpih', 'duration_ms': 201080, 'id': '4RL77hMWUq35NYnPLXBpih', 'energy': 0.29, 'time_signature': 4, 'loudness': -8.485, 'liveness': 0.118, 'danceability': 0.379, 'acousticness': 0.952, 'uri': 'spotify:track:4RL77hMWUq35NYnPLXBpih', 'key': 4, 'valence': 0.176, 'instrumentalness': 0.00106}]


In [33]:
def related_artists(name):
    related = sp.artist_related_artists(get_uri(name))
    rels = []
    for artist in related['artists']:
        rels.append(artist['name'])
    return(rels)


def next_artist(artist, artists = []):
    ''' artists is the storage of past people that have been in there'''
    RelatedArtists = related_artists(artist)
    relates = [] # empty list to append with future 
    
    for i in RelatedArtists:
        relates.append(related_artists(i) )
    
    # create a bag of words - to index each artist in:
    
    BOW = np.unique(relates).flatten() 
    vecs = np.zeros((len(relates), len(BOW)))  # unique artists as columns...

    for i in range(len(relates)):
        # turn each into a vector of 0 and 1
        for art in relates[i]:
            # this loops over each artist:
            idx = np.where(BOW == art)[0][0]
            # set that idx number = to 1
            vecs[i,idx] = 1
    
    
    # then calculate cosine similarity and select the lowest similarity artist and print them
    sims = np.zeros((len(relates),len(relates)))
    sim_means = []
    for i in range(len(vecs)):
        # loop over each vector
        for j in range(len(vecs)):
            # do this again because you want to get similarity between each matrix...
            sims[i,j] = 1 - spatial.distance.cosine(vecs[i],vecs[j])
        # calculate row wise means:
        sim_means.append(np.var(sims[i] ))
    
    # find the lowest scoring similarity artist
    lowest_idx = np.where(sim_means == max(sim_means) )[0][0]
    #print(lowest_idx)
    # return artists name:
    

    #if (RelatedArtists[lowest_idx] in artists) == True :
    # do the process again...
    
    
    print(RelatedArtists[lowest_idx])
    
    
    return(RelatedArtists[lowest_idx])

In [44]:
next_artist('Firehouse')

White Lion


'White Lion'